# Realtime Depression Detection Model using CNN and realtime images

#### Depression has been a common syndrome for most of the people in today's fast growing generation. So to spread awareness and to grow intellectuality in this domain, we are trying here to prototype a model which gives us the status and level of depression a person is facing by getting a realtime video streaming, capturing the patient's facial expressions and extracting out features from the set of images and process the set of realtime images through our CNN model to classify these into different categories such as **Happy, Sad, Fear, Disgust, Surprise, Neutral, Angry.** 
#### Post classification, we are trying to implement a logic that extracts the percentage of images falling under all of the above mentioned categories and generating a logic from those percentages and converting the values into a linear scale and classifying the results into 5 types of studied depressions mentioned below:


*   **Minimal Depression**
*   **Mild Depression**
*   **Moderate Depression**
*   **Moderately Severe Depression**

*   **Severe Depression**









### Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from zipfile import ZipFile
from tensorflow import keras
from tensorflow.keras import layers, models
import glob
import random

### Extracting files from zip arhives

In [ ]:
file_name = 'Image.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


### Reading sample image from training dataset

In [ ]:
img_array = cv2.imread('/content/Train/angry/Training_10118481.jpg')

### Getting size and dimension of RGB images

In [ ]:
img_array.shape  # These images are RGB images

### training dataset directory

In [ ]:
Data_directory = "/content/Train"  # Training Dataset

### Labels or Classes that will be used to classify the images by CNN model

In [ ]:
classes = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"] # These classes should be folder names

### Reading and displaying the images

In [ ]:
for category in classes:
  path = os.path.join(Data_directory, category)
  for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path, img))
    plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
    plt.show()
    break
  break

### Preffered size of image for our CNN model

In [ ]:
img_size = 224  # ImageNet = 224 x 224

### Resizing the realtime images to preferred image size

In [ ]:
new_array = cv2.resize(img_array, (img_size, img_size))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()

### Confirming shape of the images

In [ ]:
new_array.shape

### Read all the images and convert them into array

In [ ]:
training_data = []  ## Training Data

def create_training_data():
  for category in classes:
    path = os.path.join(Data_directory, category)
    class_num = classes.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img))
        new_array = cv2.resize(img_array, (img_size, img_size))
        training_data.append([new_array, class_num])
      except Exception as e:
        pass

### Creating Training Dataset for CNN Model

In [ ]:
create_training_data()

### Randomly shuffling the images in training dataset

In [ ]:
random.shuffle(training_data)

### Extracting and assigning the features and lebels to the images present in the dataset into two different arrays

In [ ]:
X = [] # data/feature
Y = [] # label

for feature, label in training_data:
  X.append(feature)
  Y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)  # Converting it to 4 dimension

In [ ]:
X.shape

In [ ]:
# Normalize the data
X = X / 255.0   # We are normalizing it into image pixels

In [ ]:
Y = np.array(Y)

In [ ]:
Y.shape

### Deep Learning Model for Training

#### Implementing a nested layer of CNN and MaxPooling layers
#### Using maximum 96 filters to extract and classify features from the images of training dataset to get more clear insights

In [ ]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),     # Creating a pattern of CNN and MaxPooling layers to reduce feature dimensionality as it reduces the computational cost by reducing the number of parameters to learn
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=96, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

### Debugging the code before running the model

In [ ]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Running the model for 10 epochs

In [ ]:
cnn.fit(X, Y, epochs=10)

## We are securing 96.51% of accuracy for our CNN model

In [ ]:
cnn.summary()

### Saving our CNN model 

In [ ]:
cnn.save('Face_CNN_Model.h5')

### Loading the saved CNN model

In [ ]:
new_model = tf.keras.models.load_model('/content/Face_CNN_Model.h5')

### Below we are taking some sample realtime images for testing the model

In [ ]:
frame = cv2.imread('/content/I3.jpg')  # insert here a live image

In [ ]:
frame.shape

(720, 1080, 3)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

### Below XML file is used to extract the front part of our facial structure

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

### Face Detection Logic

In [ ]:
faces = faceCascade.detectMultiScale(gray, 1.1, 4)
for x,y,w,h in faces:
  roi_gray = gray[y:y+h, x:x+w]
  roi_color = frame[y:y+h, x:x+w]
  cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)  # BGR
  face_str = faceCascade.detectMultiScale(roi_gray)
  if len(face_str) == 0:
    print("Face not detected")
  else:
    for (ex, ey, ew, eh) in face_str:
      face_roi = roi_color[ey:ey+eh, ex:ex+ew]

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))

### Resizing the image size and normalizing the image pixels

In [ ]:
final_image = cv2.resize(face_roi, (224, 224))
final_image = np.expand_dims(final_image, axis=0) 
final_image = final_image/255.0   # Normalizing the image

### Predicting the realtime images using our CNN model

In [ ]:
predictions = new_model.predict(final_image)

1/1 [==============================] - 0s 46ms/step


### Getting the predicted scores for each category, the category that ranks with higher score is the identity of that image

In [ ]:
predictions[0]

array([7.3814020e-03, 1.5741108e-11, 4.6223597e-08, 8.1112850e-01,
       1.7582282e-01, 1.1609286e-04, 5.5510793e-03, 0.0000000e+00,
       1.0324916e-35, 1.6907600e-33], dtype=float32)

### Getting the maximum score's **index**

In [ ]:
np.argmax(predictions)

3

## To get the category of image

In [ ]:
print("Facial emotion status of the image : ")
classes[np.argmax(predictions)]

Facial emotion status of the image : 


'happy'

### Load the model and test the image

In [ ]:
Data_dir = '/content/Image/'

In [ ]:
ext = ['png', 'jpeg', 'jpg']    # Add image formats here
files = []
[files.extend(glob.glob(Data_dir + '*.' + e)) for e in ext]
images = [cv2.imread(file) for file in files]
print(len(images))

77


In [ ]:
gray_image = []
for i in range(len(images)):
  gray_image.append(cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY))

In [ ]:
def face_detection(frame, gray):
  faces = faceCascade.detectMultiScale(gray, 1.1, 4)
  for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)  # BGR
    face_str = faceCascade.detectMultiScale(roi_gray)
    if len(face_str) == 0:
      print("Face not detected")
    else:
      for (ex, ey, ew, eh) in face_str:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]

  

In [ ]:
new_img_array = []
for i in range(len(gray_image)):
  face_detection(images[i], gray_image[i])
  new_img_array.append(images[i])

Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected
Face not detected


In [ ]:
for i in range(len(new_img_array)):
  plt.imshow(cv2.cvtColor(new_img_array[i], cv2.COLOR_BGR2RGB))

In [ ]:
for i in range(len(new_img_array)):
    new_img_array[i] = cv2.resize(new_img_array[i], (224, 224))
    new_img_array[i] = np.expand_dims(new_img_array[i], axis=0) 
    new_img_array[i] = new_img_array[i] / 255.0

In [ ]:
new_pred = []
for i in range(len(new_img_array)):
  new_pred.append(new_model.predict(new_img_array[i]))

1/1 [==============================] - 0s 42ms/step


In [ ]:
pred_scores = []
for i in range(len(new_img_array)):
  pred_scores.append(new_pred[i][0])

In [ ]:
pred_score_index = []
for i in range(len(new_img_array)):
  pred_score_index.append(np.argmax(pred_scores[i]))

In [ ]:
img_count = 0
pred_sentiment = []
for i in range(len(pred_score_index)):
  pred_sentiment.append(classes[pred_score_index[i]])

In [ ]:
print(pred_sentiment)

['sad', 'angry', 'fear']


In [ ]:
ac = 0  # angry count
dc = 0  # disgust count
fc = 0  # fear count
hc = 0  # happy count
nc = 0  # neutral count
sac = 0 # sad count
suc = 0 # surprise count

In [ ]:
for i in range(len(pred_sentiment)):
  if(pred_sentiment[i] == 'angry'):
    ac += 1
  elif (pred_sentiment[i] == 'disgust'):
    dc += 1
  elif (pred_sentiment[i] == 'fear'):
    fc += 1
  elif (pred_sentiment[i] == 'happy'):
    hc += 1
  elif (pred_sentiment[i] == 'neutral'):
    nc += 1
  elif (pred_sentiment[i] == 'sad'):
    sac += 1
  elif (pred_sentiment[i] == 'surprise'):
    suc += 1

In [ ]:
print("Angry :", ac, " Disgust :", dc, " Fear :", fc, " Happy :", hc, " Neutral :", nc, " Sad :", sac, " Surprise :", suc)

Angry : 12  Disgust : 4  Fear : 4  Happy : 14  Neutral : 15  Sad : 25  Surprise : 3


In [86]:
angry_per =     round(((ac) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
disgust_per =   round(((dc) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
fear_per =      round(((fc) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
happy_per =     round(((hc) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
neutral_per =   round(((nc) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
sad_per =       round(((sac) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
surprise_per =  round(((suc) / (ac + dc + fc + hc + nc + sac + suc)) * 100, 2)
print("Angry    :", angry_per, "%\nDisgust  :", disgust_per, "%\nFear     :", fear_per, "%\nHappy    :", happy_per, "%\nNeutral  :", neutral_per, "%\nSad      :", sad_per, "%\nSurprise :", surprise_per, "%")

Angry    : 15.58 %
Disgust  : 5.19 %
Fear     : 5.19 %
Happy    : 18.18 %
Neutral  : 19.48 %
Sad      : 32.47 %
Surprise : 3.9 %


### Logic for Depression from CNN model

In [87]:
depression_level = (angry_per + disgust_per + fear_per + sad_per) / 4
depression_level

14.6075

In [91]:
if(depression_level >= 1.0 and depression_level <= 4.9):
  print("Minimal Depression")
elif(depression_level >= 5.0 and depression_level <= 9.9):
  print("Mild Depression")
elif(depression_level >= 10.0 and depression_level <= 14.9):
  print("Moderate Depression")
elif(depression_level >= 15.0 and depression_level <= 19.9):
  print("Moderately Severe Depression")
elif(depression_level >= 20.0):
  print("Severe Depression")

Moderate Depression
